In [1]:
import numpy as np

from bokeh.io import show, output_notebook
from bokeh.plotting import figure
from bokeh.layouts import row, column

from scipy.integrate import quad
from scipy.special import sici

output_notebook(hide_banner=True)

# Intégration d'une fonction oscillante

On considére la fonction :

$$f: ]0,1] \to [-1,1], \ f(x) = \sin(1/x)$$

L'objectif de cet exercice est d'étudier différentes manières de calculer numériquement l'intégrale :

$$I =\displaystyle  \int_0^1 f(x)dx$$

In [2]:
def f(x):
    return np.sin(1/x)

In [5]:
x = np.linspace(1.e-10, 1, 100000)
fig = figure(width=990, height=400)
fig.line(x, f(x))
show(fig)

**Question 1 : Utiliser la fonction *quad* de *scipy.integrate* pour essayer d'intégrer directement la fonction $f$ sur $]0,1]$. Expliquer les messages d'erreur.**

In [6]:
r = quad(f,0,1)
print("integrale : ",r[0])
print("erreur : ",r[1])

integrale :  0.5024347471020806
erreur :  0.000617456903175162


C:\Users\hp\Miniconda3\envs\map412\lib\site-packages\ipykernel_launcher.py:1: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.
  """Entry point for launching an IPython kernel.


**Commentaire :**  
Le message d'erreur renvoyé montre que scipy.integrate.quad() a atteint le nombre maximal de ses itération (50) sans aboutir à une erreur satisfaisante. Ceci s'explique par les fortes oscillations de la fonction $f$ au voisinage de 0.

On note $a_k$ la suite décroissante des zéros de $f$.

**Question 2 : Utiliser *quad* de *scipy.integrate* sur chacun des intervalles $[a_{k+1},a_k]$ pour $k \leq N$. Trouver un critère d'arrêt pour avoir la précision désirée.**

In [7]:
#Les zeros de f sont a_k = 1/(k*pi)
pi = np.pi

#Critères d'arrêt : 
N = 3000
epsilon = 1e-12

# Integrale de f sur [a_1, 1]
If = quad(f,1/pi,1)
err = If[1]
If = If[0]

# Integrale sur [0, 1]
reste = 1
k = 1
while (k<N+1 and reste>epsilon) :
    t = quad(f, 1/((k+1)*pi), 1/(k*pi))
    reste = abs(t[0])
    If += t[0]
    err += t[1]
    k += 1

err += reste
print("Integrale de f sur [0,1] : I(f) = ",If)
print("Erreur : ",err)
print("Convervence pour ",k-1," itérations.")


Integrale de f sur [0,1] : I(f) =  0.5040670731573337
Erreur :  2.2596642279119242e-08
Convervence pour  3000  itérations.


**Commentaire :**  
Les zéros de f sont donnés par $\qquad \forall k \geq 1, \quad a_k = \frac{1}{k\pi}$  
On note pour $k\geq 0$ : $ I_k = \int_{a_{k+1}}^{a_k}f(t)dt$ , avec $a_0 = 1$  
On a $I(f) = \sum_{k\geq 0}I_k$. On peut montrer que la suite $(I_k)_k$ est une suite alternée, par conséquent, on a bien la convergence de $I(f)$, mais on a aussi la majoration $\forall k\geq 1 : |\sum_{j>k}I_j|\leq |I_k|$, ce qui fournit une majoration de l'erreur comise en arrêtant le calcul au $k^{ème}$ terme.

**Question 3 : Évaluez cette expression à l'aide de *scipy* et comparez la précision obtenue aux résultats des questions 1 et 2. Expliquez l'amélioration de la précision par rapport à la question 1.**

In [8]:
r = quad(lambda x : 2*x*np.cos(1/x), 0, 1)
I = np.cos(1)-r[0]
print("Integrale de f sur [0,1] : ",I)
print("Erreur : ",r[1])

Integrale de f sur [0,1] :  0.504067349270054
Erreur :  1.832793856477327e-06


C:\Users\hp\Miniconda3\envs\map412\lib\site-packages\ipykernel_launcher.py:1: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.
  """Entry point for launching an IPython kernel.


**Commentaire :**  
Le calcul s'arrête encore une fois en atteignant le nombre maximal d'itérations. Néanmoins, l'erreur obtenue est de l'orde de $10^{-6}$ alors que le calcul direct donne une erreur de $6.10^{-4}$. Ceci s'explique par la possibilité de prolonger la fonction $2x\cos (1/x)$ en une fonction continue sur $[0, 1]$, contrairement à $sin(1/x)$ qui n'admet pas de limite en 0.

**Question 4: Utiliser la fonction *sici* de *scipy.special* pour obtenir la valeur $I$ avec précision machine, en supposant que la fonction *sici* est exacte.**

In [9]:
r = sici(1)
I = np.sin(1)-r[1]
print("Valeur exacte de l'intégrale calculée avec scipy.special.sici : ",I)

Valeur exacte de l'intégrale calculée avec scipy.special.sici :  0.5040670619069283


**Question 5 (bonus) : Utiliser *quad* de *scipy.integrate* pour calculer l'intégrale de $h$ sur le chemin $\gamma_1$.
Comparer le résultat obtenu au résultat trouvé au point 2. Représenter graphiquement la fonction $\text{Im}(h)$ sur le chemin $\gamma_1$ et expliquer pourquoi *scipy.integrate* arrive à bien intégrer $h$ sur le chemin $\gamma_1$.**

In [32]:
def h(z) :
    return -np.exp(1j/z)

def gamma1(t) :
    return t+1j*t*(1-t)

def dgamma1(t) :
    return 1+1j*(1-2*t)

def im_h_sur_gamma1(t) :
    return np.imag( h(gamma1(t))*dgamma1(t) )

In [40]:
I = quad(im_h_sur_gamma1, 0, 1)
I = I[0]
print(I)

nan


C:\Users\hp\Miniconda3\envs\map412\lib\site-packages\ipykernel_launcher.py:2: RuntimeWarning: overflow encountered in exp
  
C:\Users\hp\Miniconda3\envs\map412\lib\site-packages\ipykernel_launcher.py:11: RuntimeWarning: invalid value encountered in cdouble_scalars
  # This is added back by InteractiveShellApp.init_path()
C:\Users\hp\Miniconda3\envs\map412\lib\site-packages\ipykernel_launcher.py:1: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  """Entry point for launching an IPython kernel.


In [39]:
t = np.linspace(0.001,1,100)
imh = im_h_sur_gamma1(t)
fig = figure(width=600, height=400)
fig.line(t, imh)
show(fig)

**Commentaire :**  
Les fonctions $h$ et $\gamma_1$ que j'ai implémentées correspondent bien aux définitions données sur la feuille de la PC. Toutefois, comme $\gamma_1(0)=0$, une singularité se présente au voisinage de 0 pour $h(\gamma_1(t))$, et il paraît le calcul de l'integrale ne s'effectue pas correctement en utilisant la fonction quad de scipy.integrate

# Autour de la méthode des trapèzes

**Question 1 : Première estimation d'erreur.  
Utiliser cette méthode pour calculer de manière approchée l'intégrale de $f:t\mapsto t^3 \ln t$ entre $1$ et $2$. En comparant avec la valeur donnée par la fonction *quad* de scipy, vérifier qu'on a bien une vitesse de convergence en $1/n^2$.**

In [41]:
def I_trapezes(f,a,b,N) :
    ak = np.linspace(a,b,N+1)
    h = (b-a)/N
    I = -(f(a)+f(b))/2
    for i in range(N+1) :
        I += f(ak[i])
    return h*I

In [42]:
def f(t) :
    return t*t*t*np.log(t)

In [43]:
# Calcul de l'integral avec quad de scipy :
Iquad = quad(f,1,2)
Iquad = Iquad[0]

# Calcul de l'integrale avec la méthode des trapèzes :
N = 1000
pas = 10
entiers = np.array([n for n in range(pas,N,pas)])
I = np.array([I_trapezes(f,1,2,n) for n in entiers])

# Calcul de l'erreur :
err = abs(I-Iquad)

# Figure
constante = abs(I[0]-Iquad)*entiers[0]**2
fig = figure(width=490, height=300, x_axis_type="log", y_axis_type="log")
fig.line(entiers, constante/(entiers*entiers), color = "orange")
fig.x(entiers, err, legend="|I(f)-In(f)|")

print("La valeur de l'integrale recherchée est I(f) = ",Iquad)

show(fig)

La valeur de l'integrale recherchée est I(f) =  1.8350887222397811


**Commentaire :**  
J'ai calculé les termes de la suite $I_n(f)$ pour $n\leq N$ avec un pas $pas$, ensuite j'ai représenté l'erreur $|I(f) - I_n(f)|$ en fonction de $n$ sur une échelle logarithmique.  
Enfin, le résultat théorique $|I(f) - I_n(f)| \leq \frac{(b-a)^3}{12n^2}||f''||_{\infty}$ suggère que la vitesse de convergence est au moins en $1/n^2$. Pour vérifier ce résultat, j'ai tracé sur la même figure la fonction $C/n^2$, où $C = I_{n_0}(f)n_0^2$ est une constante qui sert uniquement à faire coincider les premiers points des deux tracés. (On se permet de multiplier par $C$ car, sur l'echelle logarithmique, cette opération ne change pas la pente.  
On observe alors que les deux courbes sont identiques, ce qui montre que la vitesse de convergence est exactement $1/n^2$.

**Question 2 : Utilisation de la formule d'Euler-Maclaurin.  
Appliquer la méthode des trapèzes pour approcher l'intégrale de $g:t\mapsto t^3\ln t - \frac{3+12\ln 2}{2}(t-1)^2$ entre $1$ et $2$, et étudier la vitesse de convergence. Commenter.**

In [44]:
def g(t) :
    return t*t*t*np.log(t)-0.5*(3+12*np.log(2))*(t-1)*(t-1)

In [45]:
# Calcul de l'integral avec quad de scipy :
Iquad = quad(g,1,2)
Iquad = Iquad[0]

# Calcul de l'integrale avec la méthode des trapèzes :
N = 80
pas = 4
entiers = np.array([n for n in range(pas,N,pas)])
I = np.array([I_trapezes(g,1,2,n) for n in entiers])

# Calcul de l'erreur :
err = abs(I-Iquad)

# Figure :
fig = figure(width=490, height=300, x_axis_type="log", y_axis_type="log")

colors = ["red","orange","yellow","green"]
for m in range(2,6) :
    constante = abs(I[0]-Iquad)*entiers[0]**m
    fig.line(entiers, constante/(entiers**m), color = colors[m-2], legend=("1/(n^"+str(m)+")"))
fig.x(entiers, err, legend="|I(g)-In(g)|")
fig.legend.location = "bottom_left"

print("La valeur de l'integrale recherchée est I(g) = ",Iquad)

show(fig)

La valeur de l'integrale recherchée est I(g) =  -0.05120563888010928


**Commentaire :**  
L'estimation théorique de l'erreur établie dans l'exercice 2 montre que la vitesse de convergence est au moins en $1/n^2$, mais elle ne la précise pas. Pour étudier la vistesse de convergence, nous allons suivre deux démarches :

**1/ Méthode numérique**  
Pour obtenir la vitesse de convergence de $|I(g)-I_n(g)|$, j'ai tracé courbes $C_m/n^m$ pour $2\leq m \leq 5$, où $C_m$ est une constante qui a pour but de faire coincider les premiers points de tous ces tracés avec le premier point $(n_0, I_{n_0}(g))$ où on a estimé $I(g)$.  
La figure obtenue ci-dessus montre que la convergence est en $1/n^4$. Prouvons ce résultat théoriquement.

**2/ Méthode analytique**  
La formule d'Euler-Maclaurin permet d'écrire :

$$ I_n(g)-I(g) = -\frac{(b-a)^2}{12n^2}(g'(a)-g'(b)) + \frac{(b-a)^2}{720n^2}(g^{(3)}(a)-g^{(3)}(b)) + O( 1/n^6 )$$

Cette formule aussi montre que la convergence est au moins en $1/n^2$, mais elle nous montre en plus que si $g'(a) = g'(b)$ alors la convergence est au moins en $1/n^4$ (exactement en $1/n^4$ si $g^{(3)}(a) \ne g^{(3)}(b)$ )  
On a $\forall t\in ]0,+\infty[ : g'(t) = t^2(3ln(t)+1) - 3(1+4ln(2))(t-1)$  
Donc $g'(1) = g'(2) = 1$, d'où le résultat.

**Question 3 : Accélération de convergence et extrapolation de Richardson. 
Calculer $R^{(1)}_n$ et $R^{(2)}_n$ pour l'exemple de la question 1 ($f(t)$) et observer la vitesse de convergence.**

On trouve pour tout entier $n$ qui est une puissance de 2 :
$$ R_n^{(1)} = \frac{4I_n(f)-I_{\frac{n}{2}}(f)}{3}; \quad R_n^{(2)} = \frac{16R_n(f)-R_{\frac{n}{2}}(f)}{15}$$

In [46]:
def f(t) :
    return t*t*t*np.log(t)

In [47]:
# Calcul de l'integral avec quad de scipy :
Iquad = quad(f,1,2)
Iquad = Iquad[0]

# Calcul de l'integrale avec la méthode des trapèzes :
N = 5
entiers = np.array([2**n for n in range(N+1)])
I = np.array([I_trapezes(f,1,2,n) for n in entiers])
R1 = np.array([(4*I[1]-I[0])/3]+[(4*I[k]-I[k-1])/3 for k in range(1,N+1)]) #I[k] = I_{2^k}(f)
R2 = np.array([(16*R1[k]-R1[k-1])/15 for k in range(len(R1))])
# Calcul de l'erreur :
err1 = abs(R1[1:-1]-Iquad)
err2 = abs(R2[2:]-Iquad)

# Figure

err2 = err2*(err1[0]/err2[0])
fig = figure(width=490, height=300, x_axis_type="log", y_axis_type="log")

colors = ["red","orange","yellow","pink"]
for m in range(3,7) :
    constante = abs(R1[1]-Iquad)*entiers[0]**m
    fig.line(entiers, constante/(entiers**m), color = colors[m-3], legend=("1/(n^"+str(m)+")"))


fig.x(entiers[:-2], err1, legend="|R1-I(f)|")
fig.x(entiers[:-2], err2, color = "purple", legend="|R2-I(f)|")
print("La valeur de l'integrale recherchée est I(f) = ",Iquad)
fig.legend.location = "bottom_left"

show(fig)

La valeur de l'integrale recherchée est I(f) =  1.8350887222397811


**Commentaire :**  
Après avoir calculé numériquement err1=$|R_n^{(1)}-I(f)|$ et err2=$|R_n^{(2)}-I(f)|$, j'ai multiplié le vecteur err2 par une constante pour que les premières valeurs des deux vecteurs coincident.  
On remarque sur la figure obtenue que les vitesse de convergence pour $R_n^{(1)}$ et $R_n^{(2)}$ sont respectivement $1/n^4$ et $1/n^6$, comme prévu par la théorie.

**Question 4 (bonus) : Méthode de Romberg.  
On considère à nouveau l'exemple de la question 1 ($f(t)$). Vous n'avez le droit d'évaluer $f$ que $65$ fois maximum. Essayer d'obtenir une approximation de l'intégrale la plus précise possible. On pourra itérer le processus d'accélération décrit à la question précédente.**

**Un peu de théorie :**  
On définit la famille $(R^{(k)}_n)_{(n,k)}$ par :
$$\forall n\geq 1, R^{(0)}_{n} = I_n(f)$$
 $$\forall k\geq 1, \forall n\geq 2^k, \quad R^{(k)}_{n} = \frac{\alpha^{(k)}R^{(k-1)}_{n}-R^{(k-1)}_{n/2}}{\beta^{(k-1)}} \quad avec \quad \alpha^{(k)}=2^{2k}, \quad \beta^{(k)}=\alpha^{(k)}-1$$
La conction $f$ étant de classe $C^{\infty}$, la formule d'Euler-Maclaurin nous permet d'écrire $\forall n, I_n(f) = I(f) +C_2/n^2 + C_4/n^4 + C_6/n^6 +...$ en nous arrêtant à l'ordre qu'on veut, où $C_2,C_4,...$ sont des constantes qui ne dépendent pas ne $n$ .  
On montre alors par récurrence sur $k$ que $ R^{(k)}_{n} = I + C^{(k)}_{2k+2}/n^{2k+2} + C^{(k+1)}_{2k+4}/n^{2k+4} +...$  
Ce qui nous permet de déduire que $|R^{(k)}_{n}-I| = O(1/n^{2k+2})$

In [82]:
a = 1
b = 2
p = 6
# f sera évaluée en 2^p+1 points, pour p=6, ca fait 65 points
x = np.linspace(a, b, 2**p + 1)
valf = f(x)
R = [[0]*(p+1) for i in range(p+1)]

# R est une matrice triangulaire supérieure que nous allons remplir de haut en bas
# en utilisant la relation de récurrence établie précédemment

# Calcul de la remière ligne de R ( R[k][n] = R^(k)_(2^n) )
# Je ne vais pas utiliser la fonction I_trapzes déjà définie car sinon certaines valeurs de f seront recalculées
pasIndice = 2**p
for n in range(p+1) :
    R[0][n] = -(valf[0]+valf[-1])/2
    for j in range(2**n+1) :
        R[0][n] += valf[j*pasIndice]
    pasIndice //= 2
    R[0][n] *= (b-a)/(2**n)

# Calcul des restes des lignes de R :
for k in range(1,p+1) :
    alpha = 2**(2*k)
    beta = alpha-1
    for n in range(k, p+1) :
        R[k][n] = (alpha*R[k-1][n]-R[k-1][n-1])/beta



print ("la valeur de l'intégrale obtenue est : ",R[p][p])

la valeur de l'intégrale obtenue est :  1.8350887222397805
